In [2]:
!pip3 install ntlk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [1]:
from nltk.corpus import subjectivity

In [3]:
import nltk 

In [4]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [5]:
subjectivity.sents()[23]

['television',
 'made',
 'him',
 'famous',
 ',',
 'but',
 'his',
 'biggest',
 'hits',
 'happened',
 'off',
 'screen',
 '.']

In [6]:
subjectivity.categories()

['obj', 'subj']

In [7]:
subjectivity.words(categories='subj')

['smart', 'and', 'alert', ',', 'thirteen', ...]

Le dataset contient deux catégories d’instances: ’subj’ et ’obj’. Créer une base
d’apprentissage et une base de test contenant, respectivement, 320 et 80 instances
de chaque catégorie en utilisant subjectivity.sents(categories=...).

In [8]:
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:400]]
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:400]]
len(obj_docs), len(subj_docs)

(400, 400)

In [9]:
obj_docs[0]

(['the',
  'movie',
  'begins',
  'in',
  'the',
  'past',
  'where',
  'a',
  'young',
  'boy',
  'named',
  'sam',
  'attempts',
  'to',
  'save',
  'celebi',
  'from',
  'a',
  'hunter',
  '.'],
 'obj')

In [10]:
train_obj_docs = obj_docs[:320]
test_obj_docs = obj_docs[320:400]
train_subj_docs = subj_docs[:320]
test_subj_docs = subj_docs[320:400]

In [11]:
len(train_obj_docs)

320

In [12]:
len(test_obj_docs)

80

In [13]:
training_docs = train_obj_docs + train_subj_docs
testing_docs = test_obj_docs + test_subj_docs

In [14]:
len(training_docs)

640

Utiliser la fonction mark_negation pour ressortir les négations dans les données.

In [15]:
from nltk.sentiment import SentimentAnalyzer 
from nltk.sentiment.util import (mark_negation, extract_unigram_feats)

In [16]:
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

In [17]:
#all_words_neg

Définir un extracteur des unigrams, nommé uni_feat_extractor, en utilisant la

fonction unigram_word_feats de SentimentAnalyzer. Garder uniquement les un-
igrams qui ont une fréquence minimale égale à 5 (min_freq=5)

In [18]:
uni_feat_extractor =  sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=5)

In [19]:
#print uni_feat_extractor
len(uni_feat_extractor)

290

Ajouter l’extracteur à l’objet SentimentAnalyzer() préalablement défini 1

, en util-
isant la fonction add_feat_extractor.

In [20]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=uni_feat_extractor)

Transformer les bases d’apprentissage et de test en utilisant la fonction apply_features

In [21]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)

In [22]:
#test_set  

Construire deux classifieurs: Naive Bayes et SVM.

In [23]:
from nltk.classify import NaiveBayesClassifier

In [24]:
Naive_Bayes = NaiveBayesClassifier.train
classifier_NB = sentim_analyzer.train(Naive_Bayes, training_set)

Training classifier


In [25]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.825
F-measure [obj]: 0.8133333333333334
F-measure [subj]: 0.8352941176470587
Precision [obj]: 0.8714285714285714
Precision [subj]: 0.7888888888888889
Recall [obj]: 0.7625
Recall [subj]: 0.8875


In [26]:
from sklearn.svm import LinearSVC
from nltk.classify.scikitlearn import SklearnClassifier

In [27]:

svm= SklearnClassifier(LinearSVC())
svm_classifier = sentim_analyzer.train(svm.train, training_set) 

Training classifier


Tester les classifieurs obtenu sur la base de test et afficher les métriques d’évaluation
en utilisant sentim_analyzer.evaluate(test_set)

In [28]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating SklearnClassifier results...
Accuracy: 0.79375
F-measure [obj]: 0.7843137254901962
F-measure [subj]: 0.8023952095808383
Precision [obj]: 0.821917808219178
Precision [subj]: 0.7701149425287356
Recall [obj]: 0.75
Recall [subj]: 0.8375
